#### Data

In [1]:
import pandas as pd
import numpy as np     
import json
import random
import os

from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from scipy import stats
from sklearn.model_selection import cross_val_score

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [4]:
pre = pd.read_csv('./pna_predictset.csv')


for columns in pre.columns[1:-2]:
    
    for idx in pre.index[:178]:
        
        pre.loc[idx,columns] = 0
        
        for i,char in enumerate(list(pre.iloc[idx]["sequence"])):
            
            if str(pre.iloc[idx]["sequence"])[i:i+len(columns)] == columns[::-1]:
                
                pre.loc[idx,columns] += 1
                
pre

,No.,A,T,C,G,AA,AT,AG,AC,TT,...,GG,GA,GT,GC,CC,CA,CT,CG,sequence,score
0,Secquence 1,2.0,5.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,TGTACTTCCCTAC,NaN
1,Secquence 29,1.0,9.0,1.0,3.0,0.0,1.0,0.0,0.0,4.0,...,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,TGTAGTTGTTTTCT,NaN
2,Secquence 33,3.0,1.0,7.0,2.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,CCTCGAACACCGC,NaN
3,Secquence 34,3.0,2.0,6.0,2.0,0.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,ACGCCATCAGCTC,NaN
4,Secquence 37,4.0,1.0,6.0,2.0,1.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,1.0,1.0,2.0,1.0,2.0,ACGCCAACAGCTC,NaN
5,Secquence 38,3.0,3.0,5.0,2.0,0.0,2.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,CCTACGTATCCAG,NaN
6,Secquence 39,4.0,2.0,5.0,2.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,CCTACGAATCCAG,NaN
7,Secquence 73,2.0,1.0,3.0,7.0,1.0,0.0,1.0,0.0,0.0,...,4.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,TGGGGGAAGCGCC,NaN
8,Secquence 74,1.0,2.0,3.0,7.0,0.0,1.0,0.0,0.0,0.0,...,4.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,TGGGGGTAGCGCC,NaN
9,Secquence 89,2.0,3.0,2.0,6.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,GCAGCTAGTGGTG,NaN


#### Data PreProcessing

In [5]:
pre_nnX = []
pre_nnX_sequences = []
pre_nnY = []

for idx in pre.index:
    pre_nnX_sequences += [idx]
    pre_nnX += [np.array(pre[['A', 'T', 'C', 'G', 'AA', 'AT', 'AG', 'AC', 'TT', 'TA', 'TG',
       'TC', 'GG', 'GA', 'GT', 'GC', 'CC', 'CA', 'CT', 'CG']].loc[idx].tolist()).reshape(-1)]
    pre_nnY += [0]

pre_nnX_sequences = np.array(pre_nnX_sequences)
pre_nnX = np.array(pre_nnX)
pre_nnY = np.array(pre_nnY)

pre_nnX

array([[2., 5., 5., 1., 0., 2., 0., 0., 1., 0., 1., 2., 0., 0., 1., 0.,
        2., 2., 1., 0.],
       [1., 9., 1., 3., 0., 1., 0., 0., 4., 0., 3., 1., 0., 1., 2., 0.,
        0., 0., 1., 0.],
       [3., 1., 7., 2., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 2.,
        2., 2., 1., 1.],
       [3., 2., 6., 2., 0., 0., 0., 2., 0., 1., 0., 1., 0., 1., 0., 1.,
        1., 1., 2., 2.],
       [4., 1., 6., 2., 1., 0., 0., 2., 0., 0., 0., 1., 0., 1., 0., 1.,
        1., 2., 1., 2.],
       [3., 3., 5., 2., 0., 2., 0., 1., 0., 1., 1., 1., 0., 1., 0., 1.,
        2., 1., 1., 0.],
       [4., 2., 5., 2., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1.,
        2., 1., 1., 0.],
       [2., 1., 3., 7., 1., 0., 1., 0., 0., 0., 0., 0., 4., 1., 1., 1.,
        1., 0., 0., 2.],
       [1., 2., 3., 7., 0., 1., 0., 0., 0., 0., 1., 0., 4., 1., 1., 1.,
        1., 0., 0., 2.],
       [2., 3., 2., 6., 0., 1., 0., 1., 0., 0., 2., 1., 1., 2., 2., 0.,
        0., 0., 0., 2.],
       [3., 3., 5., 2., 1., 0.

#### Model

In [6]:
import pickle

# load the saved model
with open('my_model.pkl', 'rb') as f:
    LoRe_clf = pickle.load(f)

# get result by input X and model

Y_predict = LoRe_clf.predict(pre_nnX).reshape(-1, 1)

for idx in pre.index:     
    pre.loc[idx,'score'] = Y_predict[idx][0]

# write into file

pre.to_csv('pna_prediction_output2.csv', index=False)
pre

,No.,A,T,C,G,AA,AT,AG,AC,TT,...,GG,GA,GT,GC,CC,CA,CT,CG,sequence,score
0,Secquence 1,2.0,5.0,5.0,1.0,0.0,2.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,2.0,2.0,1.0,0.0,TGTACTTCCCTAC,1.0
1,Secquence 29,1.0,9.0,1.0,3.0,0.0,1.0,0.0,0.0,4.0,...,0.0,1.0,2.0,0.0,0.0,0.0,1.0,0.0,TGTAGTTGTTTTCT,1.0
2,Secquence 33,3.0,1.0,7.0,2.0,1.0,0.0,1.0,1.0,0.0,...,0.0,0.0,0.0,2.0,2.0,2.0,1.0,1.0,CCTCGAACACCGC,1.0
3,Secquence 34,3.0,2.0,6.0,2.0,0.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,ACGCCATCAGCTC,1.0
4,Secquence 37,4.0,1.0,6.0,2.0,1.0,0.0,0.0,2.0,0.0,...,0.0,1.0,0.0,1.0,1.0,2.0,1.0,2.0,ACGCCAACAGCTC,1.0
5,Secquence 38,3.0,3.0,5.0,2.0,0.0,2.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,CCTACGTATCCAG,1.0
6,Secquence 39,4.0,2.0,5.0,2.0,1.0,1.0,1.0,1.0,0.0,...,0.0,1.0,0.0,1.0,2.0,1.0,1.0,0.0,CCTACGAATCCAG,1.0
7,Secquence 73,2.0,1.0,3.0,7.0,1.0,0.0,1.0,0.0,0.0,...,4.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,TGGGGGAAGCGCC,-1.0
8,Secquence 74,1.0,2.0,3.0,7.0,0.0,1.0,0.0,0.0,0.0,...,4.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,TGGGGGTAGCGCC,-1.0
9,Secquence 89,2.0,3.0,2.0,6.0,0.0,1.0,0.0,1.0,0.0,...,1.0,2.0,2.0,0.0,0.0,0.0,0.0,2.0,GCAGCTAGTGGTG,-1.0
